# Approach 3: Combination of Blending Images and Gaussian Filtering on Model 2

In such an approach, 2 images of the same label are blended and then Gausian Filtering is done on the image
obtained after blending.

# Testing on 200 images

At the initial step, we will define a functions- fetch_data which helps us by, taking two random images from 
the dataset and testing it

In [38]:
import json

# Public libraries
from keras.models import model_from_json
from keras.optimizers import Adam
from keras.utils import np_utils
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random, os
from sklearn.metrics import accuracy_score

# Project
import config
import helper


def fetch_data():
    DATA_SET = 'test'  # {'test', 'train', 'valid'}

    # Load images and labels
    x, y = helper.load_data(DATA_SET)
    
    ## getting 50 random images with their labels without duplicates
    
    img_test=randomList = random.sample(range(0, 12000),200)
    
    return img_test,x,y

Now, we will define a function which will apply 2 fuctions on the images-

1) Blending two images using the formula (alpha * image1) + (beta * image2) + gamma

2) Gaussian Blur with a kernel of height 5 and width 5

The model predicts the label for every image. At last, an accuracy score is calculated.

In [43]:
def image_process():
    y_true=[]
    y_pred=[]
    model_file = config.MODEL_DEFINITION
    with open(model_file, 'r') as jfile:
        model = model_from_json(json.loads(jfile.read()))
        # Compile model and load weights
    model.compile(optimizer=Adam(), loss='categorical_crossentropy',metrics=['accuracy'])
    model.load_weights(config.MODEL_WEIGHTS)

    img_test,x,y=fetch_data()
    
    for i in range(0,200,1):
        A=x[img_test[i]]
        y_true.append(y[img_test[i]])
        
        label=str(y[img_test[i]])
        
        print("****************************************************************************")
        print("The True label is", label)
        
        if(y[img_test[i]]>9):
            label="000"+str(y[img_test[i]][0])
        else:
            label="0000"+str(y[img_test[i]][0])
    
        path_for_B=r"/home/pulkit/Downloads/GTSRB/Training/"+label+"/"
        my_image_B = random.choice([
        x for x in os.listdir(path_for_B)
        if os.path.isfile(os.path.join(path_for_B, x))
        ])
        path_B=os.path.join(path_for_B, my_image_B)
        
        B = cv2.imread(path_B,1)
        
        
        alpha=0.5
        beta=0.3
        gamma=0.2
        
        A = np.asarray(A)
        A = cv2.resize(A, (32, 32))
        
        B = np.asarray(B)
        B = cv2.resize(B, (32, 32))
        
        # blending two images of the same label
        output=cv2.addWeighted(A,alpha,B,beta,gamma)
        
        # applying gaussian fiter on the blended image
        output = cv2.GaussianBlur(output,(5,5),0)
        #print(output.shape)
        output.shape=1,32,32,3
        
        
        pred=int(model.predict_classes(output))
        print("The predicted label is ",pred)
        y_pred.append(pred)
                 
    print("****************************************************************************")        
    correct_percent=accuracy_score(y_true, y_pred)
    print("The accuracy score is ",correct_percent)
        
    
    

In [44]:
image_process()

****************************************************************************
The True label is [12]
The predicted label is  12
****************************************************************************
The True label is [12]
The predicted label is  40
****************************************************************************
The True label is [17]
The predicted label is  17
****************************************************************************
The True label is [2]
The predicted label is  2
****************************************************************************
The True label is [11]
The predicted label is  11
****************************************************************************
The True label is [2]
The predicted label is  2
****************************************************************************
The True label is [12]
The predicted label is  12
****************************************************************************
The True label is [10]
The predicted l

The predicted label is  38
****************************************************************************
The True label is [8]
The predicted label is  8
****************************************************************************
The True label is [38]
The predicted label is  38
****************************************************************************
The True label is [14]
The predicted label is  33
****************************************************************************
The True label is [11]
The predicted label is  11
****************************************************************************
The True label is [10]
The predicted label is  10
****************************************************************************
The True label is [31]
The predicted label is  31
****************************************************************************
The True label is [5]
The predicted label is  5
****************************************************************************
The True la

The predicted label is  11
****************************************************************************
The True label is [35]
The predicted label is  35
****************************************************************************
The True label is [40]
The predicted label is  40
****************************************************************************
The True label is [4]
The predicted label is  4
****************************************************************************
The True label is [10]
The predicted label is  10
****************************************************************************
The True label is [11]
The predicted label is  11
****************************************************************************
The True label is [8]
The predicted label is  8
****************************************************************************
The True label is [1]
The predicted label is  1
****************************************************************************
The True labe

The predicted label is  1
****************************************************************************
The True label is [8]
The predicted label is  8
****************************************************************************
The True label is [26]
The predicted label is  26
****************************************************************************
The True label is [9]
The predicted label is  9
****************************************************************************
The accuracy score is  0.9


Conclusion-: After blending two images of the same label and applying Gausinan Fitering,we can observe that the
prediction accuracy of the model is decreased from 95.5% to 90%.Therefore, this approach is not very
successful in breaking this model. In order to further decrease the the prediction accuracy, we can apply 
the Gaussian Filtering once more or we can do more blending on already filtered image.Further applying 
Gaussian Filtering or blending the image will make the quality of the image so bad, that even the human eye
will not be able to recognize it.

Note-: The quality of the image obtained after FGSM attack, is far superior than obtained by the combination 
of blending images and applying gaussian filter.